# Langchain crash course

In [1]:
from secret_key import openapi_key

import os
os.environ['OPENAI_API_KEY'] = openapi_key

## LLMs

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



"Maharani's Palace"


In [4]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\n"Spice Palace: A Taste of India"'

## Prompt Templates

In [5]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"Casa del Sabor" or "La Cantina Mexicana".'

In [7]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\n"Casa de Sabores" which translates to "House of Flavors" in English.'

In [8]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [11]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)

Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-instruct in organization org-GBzolOmzUHWBjRNhBNkzVVS1 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..




1. Chicken Tikka Masala
2. Lamb Vindaloo
3. Vegetable Samosas
4. Garlic Naan Bread
5. Chana Masala (spiced chickpea curry)
6. Tandoori Chicken
7. Aloo Gobi (potatoes and cauliflower in a tomato-based sauce)
8. Palak Paneer (spinach and cottage cheese curry)
9. Biryani (fragrant rice dish with choice of chicken, lamb, or vegetable)
10. Mango Lassi (yogurt-based drink with mango)
11. Butter Chicken
12. Dal Makhani (creamy black lentil stew)
13. Malai Kofta (vegetable and cheese dumplings in a creamy sauce)
14. Gulab Jamun (deep-fried milk balls in sugar syrup)
15. Mango Kulfi (Indian-style ice cream)


#### Sequential Chain

In [12]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [13]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [14]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [16]:
chain({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Saffron Spice Bistro" ',
 'menu_items': '\n\n1. Saffron Chicken Tikka Masala\n2. Lamb Korma with Saffron Rice\n3. Saffron Shrimp Biryani\n4. Vegetable Samosas with Saffron Chutney\n5. Saffron-infused Dal Makhani\n6. Tandoori Saffron Paneer Skewers\n7. Saffron Fish Curry\n8. Saffron and Pistachio Naan Bread\n9. Saffron Rice Pudding\n10. Saffron-infused Mango Lassi.'}

# Agents

#### Wikipedia and llm-math tool

In [20]:
# install this package: pip install wikipedia
from langchain.agents import AgentType, initialize_agent, load_tools

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

In [26]:
# Let's test it out!
agent.run("When was Elon musk born?")



> Entering new AgentExecutor chain...
 It would be best to use Wikipedia for this question
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$190 billion as of March 2024, according to the Bloomberg Billionaires Index, and $195 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canad

'June 28, 1971'

## Memory

In [28]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"Casa de Sabor"


In [29]:
name = chain.run("Indian")
print(name)


"Spice Emporium" 


In [30]:
chain.memory

In [31]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [32]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-instruct in organization org-GBzolOmzUHWBjRNhBNkzVVS1 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..




"Sabores de México" (Flavors of Mexico)


In [34]:
name = chain.run("Arabic")
print(name)


Al-Majlis Al-Arabi (The Arabic Banquet)


In [35]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"Sabores de México" (Flavors of Mexico)
Human: Arabic
AI: 
Al-Majlis Al-Arabi (The Arabic Banquet)


#### ConversationChain

In [36]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [37]:
convo.run("Who won the first cricket world cup?")

" The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played at Lord's Cricket Ground in London, England. The West Indies team defeated the Australia team by 17 runs to become the first ever cricket world cup champions. The first match of the tournament was played between England and India at Lord's Cricket Ground, with England winning by 202 runs. The tournament featured eight teams and was organized by the International Cricket Council. "

In [39]:
convo.run("How much is 5+5?")

' 5+5 is equal to 10.'

In [40]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning West Indies team was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler. He also went on to captain the team in the next two cricket world cups in 1979 and 1983. The West Indies team had a strong lineup with players like Viv Richards, Gordon Greenidge, and Michael Holding. They dominated the tournament and won all their matches leading up to the final.'

In [41]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played at Lord's Cricket Ground in London, England. The West Indies team defeated the Australia team by 17 runs to become the first ever cricket world cup champions. The first match of the tournament was played between England and India at Lord's Cricket Ground, with England winning by 202 runs. The tournament featured eight teams and was organized by the International Cricket Council. 
Human: How much is 5+5?
AI:  5+5 is equal to 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning West Indies team was Clive Lloyd. He was a left-handed batsman and a medium-pace bowler. He also went on to captain the team in the next two cricket world cups in 1979 and 1983. The West Indies team had a strong lineup with players like Viv Richards, Gordon Greenidge, and Michael Holding. They dominated the tournament and won all

#### ConversationBufferWindowMemory

In [43]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and was won by the West Indies team. They beat Australia in the final by 17 runs. Interestingly, the first match of the tournament was between England and India, with England winning by 202 runs.'

In [44]:
convo.run("How much is 5+5?")

'  The answer to 5+5 is 10.'

In [46]:
convo.run("Who was the captain of the winning team?")

' I do not have enough context to answer that question. Can you provide more information about the specific team and game you are referring to?'